In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import gym

In [2]:
env = gym.make('CartPole-v0')

In [3]:
env = env.unwrapped

In [4]:
BATCH_SIZE = 32
LR = 0.01                  
EPSILON = 0.9              
GAMMA = 0.9                 
TARGET_REPLACE_ITER = 100   
MEMORY_CAPACITY = 2000

In [5]:
max_actions = env.action_space.n
max_states = env.observation_space.shape[0]
ENV_A_SHAPE = 0 if isinstance(env.action_space.sample(), int) else env.action_space.sample().shape 

In [6]:
class Net(nn.Module):
    def __init__(self, ):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(max_states, 50)
        self.fc1.weight.data.normal_(0, 0.1)   
        self.out = nn.Linear(50, max_actions)
        self.out.weight.data.normal_(0, 0.1)   

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        actions_value = self.out(x)
        return actions_value

In [7]:
class DQN(object):
    def __init__(self):
        self.eval_net, self.target_net = Net(), Net()
        self.learn_step_counter = 0                                     
        
        self.memory_counter = 0                                         
        self.memory = np.zeros((MEMORY_CAPACITY, 2*max_states + 2))     
        
        self.optimizer = torch.optim.Adam(self.eval_net.parameters(), lr=LR)
        self.loss_func = nn.MSELoss()

    def choose_action(self, x):
        x = torch.unsqueeze(torch.FloatTensor(x), 0)

        if np.random.uniform() < EPSILON:  
            actions_value = self.eval_net.forward(x)
            action = torch.max(actions_value, 1)[1].data.numpy()
            action = action[0] if ENV_A_SHAPE == 0 else action.reshape(ENV_A_SHAPE)  
        else:   
            action = np.random.randint(0, max_actions)
            action = action if ENV_A_SHAPE == 0 else action.reshape(ENV_A_SHAPE)
        return action

    def store_transition(self, s, a, r, s_):
        transition = np.hstack((s, [a, r], s_))

        index = self.memory_counter % MEMORY_CAPACITY
        self.memory[index, :] = transition
        self.memory_counter += 1

    def learn(self):
        if self.learn_step_counter % TARGET_REPLACE_ITER == 0:
            self.target_net.load_state_dict(self.eval_net.state_dict())
        
        self.learn_step_counter += 1

        
        sample_index = np.random.choice(MEMORY_CAPACITY, BATCH_SIZE)
        b_memory = self.memory[sample_index, :]
        b_s = torch.FloatTensor(b_memory[:, :max_states])
        b_a = torch.LongTensor(b_memory[:, max_states:max_states+1].astype(int))
        b_r = torch.FloatTensor(b_memory[:, max_states+1:max_states+2])
        b_s_ = torch.FloatTensor(b_memory[:, -max_states:])

        q_eval = self.eval_net(b_s).gather(1, b_a)  
        q_next = self.target_net(b_s_).detach()     
        q_target = b_r + GAMMA * q_next.max(1)[0].view(BATCH_SIZE, 1)   
        loss = self.loss_func(q_eval, q_target)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

In [8]:
dqn = DQN()

In [9]:
for i_episode in range(400):
    s = env.reset()
    ep_r = 0
    
    while True:
        a = dqn.choose_action(s)

        s_, r, done, info = env.step(a)
        x, x_dot, theta, theta_dot = s_
        
        r1 = (env.x_threshold - abs(x))/env.x_threshold - 0.8
        r2 = (env.theta_threshold_radians - abs(theta)) / env.theta_threshold_radians - 0.5
        r = r1 + r2

        dqn.store_transition(s, a, r, s_)

        ep_r += r
        if dqn.memory_counter > MEMORY_CAPACITY:
            dqn.learn()
            if done:
                print('Episode:', i_episode, ', Episode Reward:', round(ep_r, 2))

        if done:
            break
        s = s_

Episode:  205 , Episode Reward:  1.23
Episode:  206 , Episode Reward:  4.81
Episode:  207 , Episode Reward:  13.67
Episode:  208 , Episode Reward:  2.55
Episode:  209 , Episode Reward:  1.38
Episode:  210 , Episode Reward:  1.94
Episode:  211 , Episode Reward:  1.08
Episode:  212 , Episode Reward:  5.28
Episode:  213 , Episode Reward:  5.95
Episode:  214 , Episode Reward:  3.03
Episode:  215 , Episode Reward:  2.45
Episode:  216 , Episode Reward:  2.6
Episode:  217 , Episode Reward:  6.43
Episode:  218 , Episode Reward:  2.11
Episode:  219 , Episode Reward:  18.12
Episode:  220 , Episode Reward:  9.74
Episode:  221 , Episode Reward:  28.43
Episode:  222 , Episode Reward:  54.87
Episode:  223 , Episode Reward:  15.22
Episode:  224 , Episode Reward:  93.72
Episode:  225 , Episode Reward:  58.09
Episode:  226 , Episode Reward:  106.95
Episode:  227 , Episode Reward:  174.89
Episode:  228 , Episode Reward:  183.86
Episode:  229 , Episode Reward:  73.81
Episode:  230 , Episode Reward:  65.3